# Data Augumentation

In [2]:
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from pprint import pprint
import pandas as pd
from tqdm.notebook import tqdm
from huggingface_hub import login

In [2]:
login()

In [3]:
sample_ds=load_dataset("s-nlp/paradetox", split="train")
sample_ds

Dataset({
    features: ['en_toxic_comment', 'en_neutral_comment'],
    num_rows: 19744
})

In [4]:
device="cuda" if torch.cuda.is_available() else "cpu"
checkpoint="dphn/Dolphin3.0-Llama3.1-8B"
model=AutoModelForCausalLM.from_pretrained(checkpoint, dtype=torch.bfloat16, device_map="auto").to(device)
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
system_prompt="You are a paraphrasing agent. You paraphrase the sentences by using maintaining same tone and same level of toxicity"
def generate_new_sentence(ex_sentence):
    prompt=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": ex_sentence}
    ]
    tokenized_prompt=tokenizer.apply_chat_template(prompt, add_generation_prompt=True, tokenize=True, return_tensors="pt").to(device)
    input_length=tokenized_prompt.shape[1]
    attention_mask = torch.ones_like(tokenized_prompt)
    output_tensors=model.generate(tokenized_prompt,
                                  attention_mask=attention_mask,
                                  pad_token_id=tokenizer.eos_token_id)[0][input_length:]
    return tokenizer.decode(output_tensors, skip_special_tokens=True)

In [9]:
def generate_new_samples(sample_ds):
    lst=[]
    for data in tqdm(sample_ds, desc="Generating new samples."):
        dct={}
        for k,v in data.items():
            dct[k]=generate_new_sentence(v)
        lst.append(dct)
    return lst

In [ ]:
new_samples=generate_new_samples(sample_ds)

Generating new samples.:   0%|          | 0/19744 [00:00<?, ?it/s]

In [ ]:
pprint(new_samples)

In [ ]:
df=pd.DataFrame(new_samples)

In [ ]:
ds=Dataset.from_pandas(df, preserve_index=False)
ds
ds=concatenate_datasets([sample_ds, ds])
ds

In [ ]:
ds.push_to_hub("harsha2946/ParaDetox-Augmented")

# ASAD

In [3]:
sample_ds=load_dataset("s-nlp/paradetox", split="train[:10]")
sample_ds

README.md: 0.00B [00:00, ?B/s]

train.tsv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/19744 [00:00<?, ? examples/s]

Dataset({
    features: ['en_toxic_comment', 'en_neutral_comment'],
    num_rows: 10
})

In [ ]:
device="cuda" if torch.cuda.is_available() else "cpu"
checkpoint="dphn/dolphin-2.9.1-llama-3-70b"
model=AutoModelForCausalLM.from_pretrained(checkpoint, dtype=torch.bfloat16, device_map="auto").to(device)
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

model-00002-of-00030.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

model-00006-of-00030.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

...

model-00012-of-00030.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

model-00013-of-00030.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00014-of-00030.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00016-of-00030.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

model-00015-of-00030.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

In [ ]:
system_prompt="You are a paraphrasing agent. You paraphrase the sentences by using maintaining same tone and same level of toxicity"
def generate_new_sentence(ex_sentence):
    prompt=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": ex_sentence}
    ]
    tokenized_prompt=tokenizer.apply_chat_template(prompt, add_generation_prompt=True, tokenize=True, return_tensors="pt").to(device)
    input_length=tokenized_prompt.shape[1]
    attention_mask = torch.ones_like(tokenized_prompt)
    output_tensors=model.generate(tokenized_prompt,
                                  attention_mask=attention_mask,
                                  pad_token_id=tokenizer.eos_token_id)[0][input_length:]
    return tokenizer.decode(output_tensors, skip_special_tokens=True)

In [ ]:
def generate_new_samples(sample_ds):
    lst=[]
    for data in tqdm(sample_ds, desc="Generating new samples."):
        dct={}
        for k,v in data.items():
            dct[k]=generate_new_sentence(v)
        lst.append(dct)
    return lst

In [ ]:
new_samples=generate_new_samples(sample_ds)

In [ ]:
new_samples